# Introduction



# Dependencies

In [ ]:
!pip3 install pandas==2.0.3 openai==0.28.1 termcolor==2.3.0 tenacity==8.2.3

# Imports

In [11]:
import json

import pandas as pd
import openai

from tenacity import retry, stop_after_attempt, wait_random_exponential
from termcolor import colored

# Data Handling

https://www.kaggle.com/datasets/yasserh/housing-prices-dataset/

In [12]:
housing_data = pd.read_csv("housing_data.csv")
print(housing_data.dtypes)
housing_data.head()

price                int64
area                 int64
bedrooms             int64
bathrooms            int64
stories              int64
mainroad            object
guestroom           object
basement            object
hotwaterheating     object
airconditioning     object
parking              int64
prefarea            object
furnishingstatus    object
dtype: object


,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [16]:
def load_housing_data():
    furn = ["unfurnished", "semi-furnished", "furnished"]
    hd = pd.read_csv("housing_data.csv")
    for col_name in ["mainroad", "guestroom", "basement", "hotwaterheating", "airconditioning", "prefarea"]:
        hd[col_name] = hd[col_name].apply(lambda x: 0 if x == "no" else 1)
    hd["furnishingstatus"] = hd["furnishingstatus"].apply(lambda x: furn.index(x))
    return hd


housing_data = load_housing_data()
print(housing_data.dtypes)
housing_data.head()

price               int64
area                int64
bedrooms            int64
bathrooms           int64
stories             int64
mainroad            int64
guestroom           int64
basement            int64
hotwaterheating     int64
airconditioning     int64
parking             int64
prefarea            int64
furnishingstatus    int64
dtype: object


,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,1,0,0,0,1,2,1,2
1,12250000,8960,4,4,4,1,0,0,0,1,3,0,2
2,12250000,9960,3,2,2,1,0,1,0,0,2,1,1
3,12215000,7500,4,2,2,1,0,1,0,1,3,1,2
4,11410000,7420,4,1,2,1,1,1,0,1,2,0,2


# OpenAI

## API Key

In [5]:
try:
    openai.api_key = open("../api_key", "r").read()
    print("API Key loaded")
except FileNotFoundError:
    print("Couldn't find the API key file")

API Key loaded


## Functions

In [6]:
def filter(housing_data:pd.DataFrame,
           condition: pd.core.series.Series):
    housing_data = housing_data[condition]
    return housing_data, len(housing_data)


def filter_lte(*args):
    housing_data = args[0]
    condition = housing_data[args[1]] <= args[2]
    return filter(housing_data, condition)


def filter_gte(*args):
    housing_data = args[0]
    condition = housing_data[args[1]] >= args[2]
    return filter(housing_data, condition)


def filter_equal(*args):
    housing_data = args[0]
    condition = housing_data[args[1]] == args[2]
    return filter(housing_data, condition)


def reset(*args):
    housing_data = load_housing_data(preprocess=True)
    return housing_data, len(housing_data)


FUNCTION_DICT = {
    "filter_lte": filter_lte,
    "filter_gte": filter_gte,
    "filter_equal": filter_equal,
    "reset": reset
}

## Function Specifications

In [7]:
FUNCTION_SPECS = [
    {
        "name": "filter_lte",
        "description": "Remove properties for which the value is more than the filter value. Return the number of remaining properties.",
        "parameters": {
            "type": "object",
            "properties": {
                "filter_name": {
                    "type": "string",
                    "description": "The characteristic to filter by.",
                },
                "filter_value": {
                    "type": "integer",
                    "description": "The maximum allowable value of the characteristic.",
                }
            },
            "required": ["filter_name", "filter_value"],
        }
    },
    {
        "name": "filter_gte",
        "description": "Remove properties for which the value is less than the filter value. Return the number of remaining properties.",
        "parameters": {
            "type": "object",
            "properties": {
                "filter_name": {
                    "type": "string",
                    "description": "The characteristic to filter by.",
                },
                "filter_value": {
                    "type": "integer",
                    "description": "The minimum allowable value of the characteristic.",
                }
            },
            "required": ["filter_name", "filter_value"],
        }
    },
    {
        "name": "filter_equal",
        "description": "Keep only properties which match the filter value. Return the number of remaining properties.",
        "parameters": {
            "type": "object",
            "properties": {
                "filter_name": {
                    "type": "string",
                    "description": "The characteristic to filter by.",
                },
                "filter_value": {
                    "type": "integer",
                    "description": "The exact value to match.",
                }
            },
            "required": ["filter_name", "filter_value"],
        }
    },
    {
        "name": "reset",
        "description": "Reset the list of houses to its original state and return the number of properties.",
        "parameters": {
            "type": "object",
            "properties": {}
        }
    }
]

## API Calls

In [8]:
class Realtor:

    def __init__(self, system_content: str, function_specs: list, functions: dict, model: str = "gpt-3.5-turbo", debug=False):
        self.model = model
        self.function_specs = function_specs
        self.functions = functions

        self.housing_data = load_housing_data()
        self.convo: list = [{"role": "system", "content": system_content.format(len(self.housing_data))}]
        self.debug = debug
        self.colours = {
            "assistant": "blue",
            "user": "black",
            "function_call": "red",
            "function_response": "green"
        }

    @retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
    def call_completion_api(self):
        completion = openai.ChatCompletion.create(
            model=self.model,
            messages=self.convo,
            functions=self.function_specs
        )

        return completion.choices[0]["message"]

    def converse(self):

        # Get the initial welcome from ChatGPT
        assistant_message = self.call_completion_api()
        self.remember(assistant_message)

        while True:
            # Get user input and add it to the convo (or quit on 'q')
            user_input = input("> ")
            if user_input == "q":
                break
            self.remember({"role": "user", "content": user_input})

            # Get ChatGPT response to user input
            assistant_message = self.call_completion_api()

            # Handle function calls until ChatGPT is happy
            while assistant_message.get("function_call"):
                # Record the function call
                self.remember(assistant_message)

                # Get result function
                function_result = self.call_function(assistant_message["function_call"])
                self.remember(function_result)

                # Send convo updated with function result to ChatGPT
                assistant_message = self.call_completion_api()

            # Remember initial response to user input or ChatGPT response to function result
            self.remember(assistant_message)

    def remember(self, data):
        colour: str = ""
        print_message: str = ""
        convo_message: str = ""
        
        if data["role"] == "assistant":
            convo_message = data.to_dict_recursive()
            if data.get("function_call"):
                # function call
                if self.debug:
                    colour = self.colours["function_call"]
                    print_message = data.function_call.to_dict_recursive()
            else:
                # assistant message
                colour = self.colours["assistant"]
                print_message = data.content
        elif data["role"] == "user":
            # user content
            convo_message = data
            colour = self.colours["user"]
            print_message = data["content"]
        else:
            # function response
            convo_message = data
            if self.debug:
                colour = self.colours["function_response"]
                print_message = data

        self.convo.append(convo_message)
        if len(print_message) > 0:
            print(colored(print_message, colour))

    def call_function(self, function_call):

        # Extract the function call info
        function_name = function_call["name"]
        function_to_call = self.functions[function_name]
        function_args = json.loads(function_call["arguments"])
        arg_values = [v for _, v in function_args.items()]

        self.housing_data, num_properties = function_to_call(
            *[self.housing_data, *arg_values]
        )

        return {"role": "function", "name": function_name, "content": str(num_properties)}

In [10]:
REALTOR_SYSTEM_CONTENT = \
"""
You are a helpful realtor with a list initially containing {} properties.
For each property, we have the following characteristics:
    price (asking price in euro),
    area (total floor area in metres squared),
    bedrooms (number of bedrooms),
    bathrooms (number of bathrooms),
    stories (number of stories),
    mainroad (1 if the property is on a main road, otherwise 0),
    guestroom (1 if the property has a guestroom, otherwise 0),
    basement (1 if the property has a basement, otherwise 0),
    hotwaterheating (1 if the property has a water heating system, otherwise 0),
    airconditioning (1 if the property has air conditioning, otherwise 0),
    parking (number of parking spaces),
    prefarea (1 if the property is in a trendy neighbourhood, otherwise 0),
    furnishingstatus (0 if unfurnished, 1 if semi-furnished, 2 if fully furnished)
Welcome the customer to the realtor site and offer to help narrow this selection based on their requirements.
If the customer changes their mind about a feature, reset the list and reapply previous filters alongside the corrected requirement.
"""

# I'm looking for a house. The budget is 5000000; we are a two car family; we need somewhere without stairs.

realtor = Realtor(REALTOR_SYSTEM_CONTENT, FUNCTION_SPECS, FUNCTION_DICT, debug=True)
realtor.converse()

Welcome to the realtor site! How can I assist you today?


>  i'm looking for something in an upcoming neighbourhood.


i'm looking for something in an upcoming neighbourhood.
Great! I can help you with that. Could you please provide me with some more details? What is your preferred location or city?


>  i'm looking for something in a hip neighbourhood.


i'm looking for something in a hip neighbourhood.
{'name': 'filter_equal', 'arguments': '{\n  "filter_name": "prefarea",\n  "filter_value": 1\n}'}
{'role': 'function', 'name': 'filter_equal', 'content': '128'}
I have narrowed down the list to 128 properties located in trendy neighbourhoods. How else can I assist you in finding your ideal property?


>  i need both aircon and 24/7 hot showers.


i need both aircon and 24/7 hot showers.
{'name': 'filter_equal', 'arguments': '{\n  "filter_name": "airconditioning",\n  "filter_value": 1\n}'}
{'role': 'function', 'name': 'filter_equal', 'content': '53'}
{'name': 'filter_equal', 'arguments': '{\n  "filter_name": "hotwaterheating",\n  "filter_value": 1\n}'}
{'role': 'function', 'name': 'filter_equal', 'content': '1'}
I have filtered the list based on your requirements for both air conditioning and 24/7 hot showers. There are currently 1 property that meets these criteria. How else can I assist you?


>  q
